In [0]:
import numpy as np
import tensorflow as tf
from __future__ import print_function
import tensorflow.keras
from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import os
from google.colab import drive
drive.mount('/content/gdrive')

# Install a Drive FUSE wrapper.
 # https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null 
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Generate auth tokens for Colab

from google.colab import auth 
auth.authenticate_user()

# Generate creds for the Drive FUSE library.

from oauth2client.client import GoogleCredentials 
creds = GoogleCredentials.get_application_default()
import getpass 
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass() 
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# Create a directory and mount Google Drive using that directory.

!mkdir -p drive
!google-drive-ocamlfuse drive 


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144433 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.21-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for ma

In [0]:
import pandas as pd 
l=pd.read_csv("drive/Colab Notebooks/valid-acl.csv")
label=l['label']
index=l['index']
dataset=[]
x=[""]

for i in index:
  dataset.append(np.load('drive/Colab Notebooks/sagittal/'+str(i)+'.npy', encoding='bytes'))


In [0]:
 dataset=np.asarray(dataset)
print(dataset[0].shape)

(27, 256, 256)


In [0]:
min=100
for i in dataset:
  if min>i.shape[0]:
    min=i.shape[0]
print(min)

21


In [0]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr



In [0]:
import numpy as np
import keras
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import np_utils
from keras.utils import plot_model
import scipy.misc
from keras.initializers import glorot_uniform
from matplotlib.pyplot import imshow
import keras.backend as K
import tensorflow as tf
from tensorflow.keras.initializers import RandomNormal as RN
from tensorflow.keras.initializers import GlorotNormal as GN
from tensorflow.keras.initializers import GlorotUniform as GU

K.set_image_data_format('channels_last')
K.set_learning_phase(1)
class ResNet50():

  def __init__(self,shape ,weightsPath = None):
    self.shape =shape;
    self.weightsPath = weightsPath;

  def identity_block(self,X,f , filters, stage, block):
    """
    Implementation of the identity block
    
    Arguments:
    X -- input tensor of shape (m, #heddin layers in prev, #W in prev, #channels in prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (#H, #W, #C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    
    # Second component of main path 
    X = Conv2D(filters = F2, kernel_size=(f,f), strides = (1,1), padding='same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path 
    X = Conv2D(filters = F3, kernel_size=(1,1), strides = (1,1), padding="valid", name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

  def convolutional_block(self,X, filters, stage, block, s =2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    #F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(filters = filters[0], kernel_size= (1, 1), strides = (s,s),padding="valid", name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    

    # Second component of main path 
    X = Conv2D(filters = filters[1], kernel_size=(3,3), strides=(1,1), name = conv_name_base + '2b', padding="same",kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name= bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path 
    X = Conv2D(filters = filters[2], kernel_size=(1,1), strides = (1,1), name= conv_name_base + '2c',padding="valid", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### 
    X_shortcut = Conv2D(filters = filters[2], kernel_size= (1,1), strides=(s,s), name=conv_name_base + '1', padding="valid", kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base+'1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X_shortcut,X])
    X = Activation("relu")(X)
    
    
    return X

  def getModel(self,classes = 1):
    """
    Arguments:
    classes -- integer, number of classes
    Returns:
    model -- a Model() instance in Keras
    """
    input = Input(shape=self.shape);
    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    st2filters=[64, 64, 256]
    X = self.convolutional_block(X,st2filters, 2, 'a', 1)
    X = self.identity_block(X, 3, [64, 64, 256], 2, 'b')
    X = self.identity_block(X, 3, [64, 64, 256], 2, 'c')


    # Stage 3
    X = self.convolutional_block(X, [128,128,512],3,'a', 2)
    X = self.identity_block(X, 3, [128,128,512], 3, 'b')
    X = self.identity_block(X, 3, [128,128,512], 3, 'c')
    X = self.identity_block(X, 3, [128,128,512], 3, 'd')

    # Stage 4 
    X = self.convolutional_block(X, [256,256,1024],4 , 'a', 2)
    X = self.identity_block(X, 3, [256, 256, 1024],4 , 'b')
    X = self.identity_block(X, 3, [256, 256, 1024],4 , 'c')
    X = self.identity_block(X, 3, [256, 256, 1024],4 , 'd')
    X = self.identity_block(X, 3, [256, 256, 1024],4 , 'e')
    X = self.identity_block(X, 3, [256, 256, 1024],4 , 'f')

    # Stage 5 
    X = self.convolutional_block(X, [256,256,2048], 5,'a', 3)
    X = self.identity_block(X, 3, [256,256,2048], 5, 'b')
    X = self.identity_block(X, 3, [256,256,2048], 5, 'c')

    # AVGPOOL (Ã¢â€°Ë†1 line). Use "X = AveragePooling2D(...)(X)"
    #X = AveragePooling2D((2,2), name='avg_pool')(X)
    

    # output layer
    #X = Flatten()(X)
    #X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = input, outputs = X, name='ResNet50')

    if self.weightsPath:
        model.load_weights(self.weightsPath)

    return model

class MRNet_ResNet_layer(keras.layers.Layer):
    
  def __init__(self, input_shape,batch_size):
    super(MRNet_ResNet_layer, self).__init__()
    self.ResNet = ResNet50(input_shape).getModel()
    self.avg_pooling = AveragePooling2D((2, 2), name='avg_pool')
    self.fc = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))
    self.b_size = batch_size

  def compute_output_shape(self, input_shape):
    return (None, 2)

  @tf.function
  def call(self, inputs):
    arr = []
    for index in range(self.b_size):
      out = self.ResNet(inputs[index])
      out = tf.squeeze(self.avg_pooling(out), axis=[1, 2])
      out = keras.backend.max(out, axis=0, keepdims=True)
      out = tf.squeeze(out)
      arr.append(out)
    output = tf.stack(arr, axis=0)
    output = self.fc(output)
    return output

  def compute_output_shape(self, input_shape):
    return (None, 1)


def scheduler(epoch):
  if epoch < 10:
    return 0.001
  else:
    return 0.001 * tf.math.exp(0.1 * (10 - epoch))

def MRNet_ResNet_model(batch_size, combination = ["abnormal", "axial"]):
  METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'), 
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
  ]

  b_size = batch_size
  model = keras.Sequential()
  model.add(MRNet_ResNet_layer(( 256, 256, 3), b_size))
  model(Input(shape=(None, 224, 224, 3)))
  model.compile(
      #optimizer=tf.keras.optimizers.Adam(lr=0.001, decay=1e-2),
      optimizer=tf.keras.optimizers.RMSprop(lr=.01),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=METRICS)
  data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/"
  checkpoint_dir = data_path+"training_ResNet/" + combination[0] + "/" + combination[1] + "/"
  # checkpoint_dir = os.path.dirname(checkpoint_path)
  if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
  os.chdir(checkpoint_dir)
  cp_callback = keras.callbacks.ModelCheckpoint(checkpoint_dir+"weights.{epoch:02d}.hdf5",
                                                 save_weights_only=True,
                                                 verbose=1)
  tcb = TestCallback(model)
  return model, [cp_callback, tcb]

class TestCallback(tf.keras.callbacks.Callback):
  def __init__(self, model):
    super(TestCallback, self).__init__()
    self.model = model

  def on_epoch_end(self, epoch, logs=None):
    if(epoch == 0):
      self.w = self.model.layers[0].get_weights()[0]
      return
    self.w_after = self.model.layers[0].get_weights()[0]
    print('  TestCallback: ', (self.w == self.w_after).all())
    self.w = self.w_after

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd "/content/gdrive/My Drive/Colab Notebooks/MRNet/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/MRNet


In [0]:
import os
import numpy as np
import glob
import csv
import keras
import scipy
from partition import partition
from data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from matplotlib.pyplot import imshow
import random
import tensorflow as tf
from numpy.random import seed
from sklearn.utils import class_weight
from ResNet_model_generator1 import MRNet_ResNet_model
# seed(1)
# from tensorflow import set_random_seed
# tf.random.set_seed(2)


Using TensorFlow backend.


In [0]:
label_type = 'ACL'
exam_view = 'coronal'
model, cbs = MRNet_ResNet_model(2,.00000005,[label_type, exam_view])
# print(model(tf.ones((12, 24, 224, 224, 3))).shape)

model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_ResNet/ACL/coronal/weights.16.hdf5")
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['train'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['train'][exam_view])
neg = total - pos
class_count_train = {0: neg, 1: pos}
pos = 0
for e in p['test'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['test'][exam_view])
neg = total - pos
class_count_test = {0: neg, 1: pos}
# print(class_count_train, class_count_test)
batch_size = 1
train_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_train , batch_size=1,model="ResNet",data_type='train', label_type=label_type, exam_type=exam_view, scale_to=(224,224), aug_size=1)
test_data_gen = MRNet_data_generator(data_path, p, l, class_count=class_count_test, batch_size=1,model="ResNet",data_type='test', label_type=label_type, exam_type=exam_view, scale_to=(224,224), aug_size=1)
# print(len(train_data_gen))
model.fit(x=train_data_gen,validation_data=test_data_gen, epochs=10, verbose=True, callbacks=cbs,shuffle=False, use_multiprocessing=True, workers=6)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Initializing Data Generator:
factor :  4.16243654822335
model:  ResNet
data type:  train
Combination:  ACL  and  coronal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/train
Number of inputs:  1639
input size:  (224, 224)
Initializing Data Generator:
factor :  9.272727272727273
model:  ResNet
data type:  test
Combination:  ACL  and  coronal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/test
Number of inputs:  204
input size:  (224, 224)
Epoch 1/10
1639/1639 [==============================] - 1261s 769ms/step - loss: 0.3977 - tp: 525.6254 - fp: 98.9695 - tn: 722.1568 - fn: 293.2483 - accuracy: 0.7624 - precision: 0.8456 - recall: 0.6435 - auc: 0.8851 - val_loss: 0.8113 - val_tp: 1072.2745 - val_fp: 216.6814 - val_tn: 1530.4854 - val_fn: 663.5588 - val_accuracy: 0.7475 - val_precision: 0.8319 - val_recall: 0.6181 - val_auc: 0.8713

Epoch 00001: s

In [0]:
%

training with the choosen model and hyperparameters

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd "/content/gdrive/My Drive/Colab Notebooks/MRNet/"

import os
import numpy as np
import glob
import csv
import keras
import scipy
from partition import partition
from data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from matplotlib.pyplot import imshow
import random
import tensorflow as tf
from numpy.random import seed
from sklearn.utils import class_weight
from ResNet_model_generator2 import MRNet_ResNet_model
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import auc as roc_auc

label_type = 'meniscus'
exam_view = 'coronal'
model, cbs = MRNet_ResNet_model(1,0.00001,[label_type, exam_view])
# print(model(tf.ones((12, 24, 224, 224, 3))).shape)

#model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_ResNet/"+label_type+"/"+exam_view+"/weights.03.hdf5")
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['train'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['train'][exam_view])
neg = total - pos
class_count_train = {0: neg, 1: pos}
pos = 0
for e in p['test'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['test'][exam_view])
neg = total - pos
class_count_test = {0: neg, 1: pos}
# print(class_count_train, class_count_test)
batch_size = 1
train_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_train , batch_size=1,model="ResNet",data_type='train', label_type=label_type, exam_type=exam_view, scale_to=(224,224),  aug_size=0)
test_data_gen = MRNet_data_generator(data_path, p, l, class_count=class_count_test, batch_size=1,model="ResNet",data_type='test', label_type=label_type, exam_type=exam_view, scale_to=(224,224), aug_size=0)
# print(len(train_data_gen))
model.fit(x=train_data_gen,validation_data=test_data_gen, epochs=5, verbose=True, callbacks=cbs,shuffle=True, use_multiprocessing=True, workers=6)

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/MRNet


Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Initializing Data Generator:
factor :  1.778688524590164
model:  ResNet
data type:  train
Combination:  meniscus  and  coronal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/train
Number of inputs:  1302
input size:  (224, 224)
Initializing Data Generator:
factor :  2.6451612903225805
model:  ResNet
data type:  test
Combination:  meniscus  and  coronal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/test
Number of inputs:  163
input size:  (224, 224)
Epoch 1/5
1302/1302 [==============================] - 782s 601ms/step - loss: 0.7128 - tp: 230.7104 - fp: 184.4685 - tn: 140.0215 - fn: 96.2995 - accuracy: 0.5625 - precision: 0.5528 - recall: 0.7458 - auc: 0.5946 - val_loss: 0.2593 - val_tp: 481.9939 - val_fp: 361.4478 - val_tn: 332.2515 - val_fn: 208.3067 - val_accuracy: 0.5884 - val_precision: 0.5715 - val_recall: 0.6985 - val_auc: 0.6394

Epoch 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd "/content/gdrive/My Drive/Colab Notebooks/MRNet/"

import os
import numpy as np
import glob
import csv
import keras
import scipy
from partition import partition
from data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from matplotlib.pyplot import imshow
import random
import tensorflow as tf
from numpy.random import seed
from sklearn.utils import class_weight
from ResNet_model_generator2 import MRNet_ResNet_model
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import auc as roc_auc

label_type = 'meniscus'
exam_view = 'coronal'
model, cbs = MRNet_ResNet_model(1, 0.000001,combination=[label_type, exam_view])
# print(model(tf.ones((12, 24, 224, 224, 3))).shape)

model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_ResNet/"+label_type+"/"+exam_view+"/weights.05.hdf5")
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['valid'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['valid'][exam_view])
neg = total - pos
class_count_valid = {0: neg, 1: pos}
batch_size = 1
valid_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_valid , shuffle=1, batch_size=1,model="ResNet",data_type='valid', label_type=label_type, exam_type=exam_view, scale_to=(224,224), aug_size=0)
# print(model.predict(valid_data_gen, batch_size=None,verbose=1))
# print(model.metrics_names)
tp = 0
tn = 0
fp = 0
fn = 0
y_true = []
y_score = []
for i in range(len(valid_data_gen)):
  x, y = next(valid_data_gen)
  _y = model.predict_classes(x, batch_size=1, verbose=0)
  _y_score = model.predict_proba(x, batch_size=2)
  y_score.append(_y_score[0][0])
  y_true.append(y[0])
  if _y[0][0] == 1:
    if _y[0][0] == y[0]:
      tp += 1
    else:
      fp += 1
  else:
    if _y[0][0] == y[0]:
      tn += 1
    else:
      fn += 1
y_score = np.array(y_score)
y_true = np.array(y_true)
fpr, tpr, _ = roc_curve(y_true, y_score)
auc = roc_auc(fpr, tpr)
print ("tp = ", tp, " fp = ", fp, " tn = ", tn, " fn = ", fn, " accuracy = ", (tp+tn)/(len(valid_data_gen)), " auc = ", auc)

# model.fit(x=train_data_gen,validation_data=test_data_gen, epochs=12, verbose=True, callbacks=cbs,shuffle=False, use_multiprocessing=True, workers=6)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/MRNet


Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Initializing Data Generator:
factor :  1.3076923076923077
model:  ResNet
data type:  valid
Combination:  meniscus  and  coronal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/valid
Number of inputs:  120
input size:  (224, 224)
tp =  20  fp =  26  tn =  42  fn =  32  accuracy =  0.5166666666666667  auc =  0.521210407239819


In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd "/content/gdrive/My Drive/Colab Notebooks/MRNet/"

import os
import numpy as np
import glob
import csv
import keras
import scipy
from partition import partition
from data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from matplotlib.pyplot import imshow
import random
import tensorflow as tf
from numpy.random import seed
from sklearn.utils import class_weight
from ResNet_model_generator2 import MRNet_ResNet_model
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import auc as roc_auc

label_type = 'meniscus'
exam_view = 'axial'
model, cbs = MRNet_ResNet_model(1,0.00001,[label_type, exam_view])
# print(model(tf.ones((12, 24, 224, 224, 3))).shape)

#model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_ResNet/"+label_type+"/"+exam_view+"/weights.10.hdf5")
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['train'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['train'][exam_view])
neg = total - pos
class_count_train = {0: neg, 1: pos}
pos = 0
for e in p['test'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['test'][exam_view])
neg = total - pos
class_count_test = {0: neg, 1: pos}
# print(class_count_train, class_count_test)
batch_size = 1
train_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_train , batch_size=1,model="ResNet",data_type='train', label_type=label_type, exam_type=exam_view, scale_to=(224,224),  aug_size=0)
test_data_gen = MRNet_data_generator(data_path, p, l, class_count=class_count_test, batch_size=1,model="ResNet",data_type='test', label_type=label_type, exam_type=exam_view, scale_to=(224,224), aug_size=0)
# print(len(train_data_gen))
model.fit(x=train_data_gen,validation_data=test_data_gen, epochs=15, verbose=True, callbacks=cbs,shuffle=False, use_multiprocessing=True, workers=6)

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/MRNet


Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Initializing Data Generator:
factor :  1.778688524590164
model:  ResNet
data type:  train
Combination:  meniscus  and  axial
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/train
Number of inputs:  1302
input size:  (224, 224)
Initializing Data Generator:
factor :  2.6451612903225805
model:  ResNet
data type:  test
Combination:  meniscus  and  axial
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/test
Number of inputs:  163
input size:  (224, 224)
Epoch 1/15
1302/1302 [==============================] - 867s 666ms/step - loss: 0.7831 - tp: 265.8518 - fp: 236.1014 - tn: 86.4585 - fn: 63.0883 - accuracy: 0.5274 - precision: 0.5173 - recall: 0.8341 - auc: 0.5442 - val_loss: 0.3613 - val_tp: 553.7791 - val_fp: 481.3865 - val_tn: 210.7362 - val_fn: 138.0982 - val_accuracy: 0.5524 - val_precision: 0.5349 - val_recall: 0.8005 - val_auc: 0.6103

Epoch 0000

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd "/content/gdrive/My Drive/Colab Notebooks/MRNet/"

import os
import numpy as np
import glob
import csv
import keras
import scipy
from partition import partition
from data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from matplotlib.pyplot import imshow
import random
import tensorflow as tf
from numpy.random import seed
from sklearn.utils import class_weight
from ResNet_model_generator2 import MRNet_ResNet_model
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import auc as roc_auc

label_type = 'meniscus'
exam_view = 'axial'
model, cbs = MRNet_ResNet_model(1, 0.00001,combination=[label_type, exam_view])
# print(model(tf.ones((12, 24, 224, 224, 3))).shape)

model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_ResNet/"+label_type+"/"+exam_view+"/weights.09.hdf5")
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['valid'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['valid'][exam_view])
neg = total - pos
class_count_valid = {0: neg, 1: pos}
batch_size = 1
valid_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_valid , shuffle=1, batch_size=1,model="ResNet",data_type='valid', label_type=label_type, exam_type=exam_view, scale_to=(224,224), aug_size=0)
# print(model.predict(valid_data_gen, batch_size=None,verbose=1))
# print(model.metrics_names)
tp = 0
tn = 0
fp = 0
fn = 0
y_true = []
y_score = []
for i in range(len(valid_data_gen)):
  x, y = next(valid_data_gen)
  _y = model.predict_classes(x, batch_size=1, verbose=0)
  _y_score = model.predict_proba(x, batch_size=2)
  y_score.append(_y_score[0][0])
  y_true.append(y[0])
  if _y[0][0] == 1:
    if _y[0][0] == y[0]:
      tp += 1
    else:
      fp += 1
  else:
    if _y[0][0] == y[0]:
      tn += 1
    else:
      fn += 1
y_score = np.array(y_score)
y_true = np.array(y_true)
fpr, tpr, _ = roc_curve(y_true, y_score)
auc = roc_auc(fpr, tpr)
print ("tp = ", tp, " fp = ", fp, " tn = ", tn, " fn = ", fn, " accuracy = ", (tp+tn)/(len(valid_data_gen)), " auc = ", auc)

# model.fit(x=train_data_gen,validation_data=test_data_gen, epochs=12, verbose=True, callbacks=cbs,shuffle=False, use_multiprocessing=True, workers=6)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/MRNet


Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Initializing Data Generator:
factor :  1.3076923076923077
model:  ResNet
data type:  valid
Combination:  meniscus  and  axial
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/valid
Number of inputs:  120
input size:  (224, 224)
tp =  13  fp =  18  tn =  50  fn =  39  accuracy =  0.525  auc =  0.5421380090497737


In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd "/content/gdrive/My Drive/Colab Notebooks/MRNet/"

import os
import numpy as np
import glob
import csv
import keras
import scipy
from partition import partition
from data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from matplotlib.pyplot import imshow
import random
import tensorflow as tf
from numpy.random import seed
from sklearn.utils import class_weight
from ResNet_model_generator2 import MRNet_ResNet_model
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import auc as roc_auc

label_type = 'meniscus'
exam_view = 'coronal'
model, cbs = MRNet_ResNet_model(1,0.00001,[label_type, exam_view])
# print(model(tf.ones((12, 24, 224, 224, 3))).shape)

#model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_ResNet/"+label_type+"/"+exam_view+"/weights.03.hdf5")
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['train'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['train'][exam_view])
neg = total - pos
class_count_train = {0: neg, 1: pos}
pos = 0
for e in p['test'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['test'][exam_view])
neg = total - pos
class_count_test = {0: neg, 1: pos}
# print(class_count_train, class_count_test)
batch_size = 1
train_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_train , batch_size=1,model="ResNet",data_type='train', label_type=label_type, exam_type=exam_view, scale_to=(224,224),  aug_size=0)
test_data_gen = MRNet_data_generator(data_path, p, l, class_count=class_count_test, batch_size=1,model="ResNet",data_type='test', label_type=label_type, exam_type=exam_view, scale_to=(224,224), aug_size=0)
# print(len(train_data_gen))
model.fit(x=train_data_gen,validation_data=test_data_gen, epochs=15, verbose=True, callbacks=cbs,shuffle=False, use_multiprocessing=True, workers=6)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/MRNet


Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Initializing Data Generator:
factor :  1.778688524590164
model:  ResNet
data type:  train
Combination:  meniscus  and  coronal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/train
Number of inputs:  1302
input size:  (224, 224)
Initializing Data Generator:
factor :  2.6451612903225805
model:  ResNet
data type:  test
Combination:  meniscus  and  coronal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/test
Number of inputs:  163
input size:  (224, 224)
Epoch 1/15
1302/1302 [==============================] - 434s 333ms/step - loss: 0.7123 - tp: 234.3318 - fp: 187.4700 - tn: 139.7565 - fn: 89.9416 - accuracy: 0.5504 - precision: 0.5390 - recall: 0.7567 - auc: 0.5538 - val_loss: 0.3664 - val_tp: 495.2883 - val_fp: 355.4478 - val_tn: 338.0797 - val_fn: 195.1841 - val_accuracy: 0.6021 - val_precision: 0.5822 - val_recall: 0.7174 - val_auc: 0.6387

Epoch

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd "/content/gdrive/My Drive/Colab Notebooks/MRNet/"

import os
import numpy as np
import glob
import csv
import keras
import scipy
from partition import partition
from data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from matplotlib.pyplot import imshow
import random
import tensorflow as tf
from numpy.random import seed
from sklearn.utils import class_weight
from ResNet_model_generator2 import MRNet_ResNet_model
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import auc as roc_auc

label_type = 'meniscus'
exam_view = 'coronal'

model, cbs = MRNet_ResNet_model(1, 0.000001,combination=[label_type, exam_view])
# print(model(tf.ones((12, 24, 224, 224, 3))).shape)

model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_ResNet/"+label_type+"/"+exam_view+"/weights.13.hdf5")
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['valid'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['valid'][exam_view])
neg = total - pos
class_count_valid = {0: neg, 1: pos}
batch_size = 1
valid_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_valid , shuffle=1, batch_size=1,model="ResNet",data_type='valid', label_type=label_type, exam_type=exam_view, scale_to=(224,224), aug_size=0)
# print(model.predict(valid_data_gen, batch_size=None,verbose=1))
# print(model.metrics_names)
tp = 0
tn = 0
fp = 0
fn = 0
y_true = []
y_score = []
for i in range(len(valid_data_gen)):
  x, y = next(valid_data_gen)
  _y = model.predict_classes(x, batch_size=1, verbose=0)
  _y_score = model.predict_proba(x, batch_size=2)
  y_score.append(_y_score[0][0])
  y_true.append(y[0])
  if _y[0][0] == 1:
    if _y[0][0] == y[0]:
      tp += 1
    else:
      fp += 1
  else:
    if _y[0][0] == y[0]:
      tn += 1
    else:
      fn += 1
y_score = np.array(y_score)
y_true = np.array(y_true)
fpr, tpr, _ = roc_curve(y_true, y_score)
auc = roc_auc(fpr, tpr)
print ("tp = ", tp, " fp = ", fp, " tn = ", tn, " fn = ", fn, " accuracy = ", (tp+tn)/(len(valid_data_gen)), " auc = ", auc)

# model.fit(x=train_data_gen,validation_data=test_data_gen, epochs=12, verbose=True, callbacks=cbs,shuffle=False, use_multiprocessing=True, workers=6)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/MRNet


Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Initializing Data Generator:
factor :  1.3076923076923077
model:  ResNet
data type:  valid
Combination:  meniscus  and  coronal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/valid
Number of inputs:  120
input size:  (224, 224)
tp =  16  fp =  21  tn =  47  fn =  36  accuracy =  0.525  auc =  0.5214932126696832
